## Article Generation Supervisor Agent

In the previous notebooks, we built two specialized agents:

1. **Article Generation Agent** (Notebook 01): An expert content creator that writes high-quality news, sports, and entertainment articles following detailed journalistic guidelines. This agent focuses on clarity, accuracy, engagement, structure, balance, and technical excellence.
2. **Article Reviewer Agent** (Notebook 02): A professional reviewer that analyzes articles across multiple dimensions including readability, factual accuracy, engagement, fairness, context, and ethical considerations. This agent provides detailed, constructive feedback with specific improvement suggestions.

In this notebook, we'll implement a **Multi-Agent Collaboration** pattern by creating a supervisor agent that orchestrates these two specialized agents to work together iteratively. This pattern enables a powerful feedback loop where content is continuously refined through expert review cycles.

## Multi-Agent Collaboration Pattern

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is an Amazon Bedrock Agents capability that enables hierarchical collaboration between agents. This pattern allows:

- A supervisor agent to coordinate specialized sub-agents
- Delegation of specific tasks to agents with specialized expertise
- Iterative improvement through feedback loops
- Complex workflows that leverage multiple AI capabilities

The supervisor agent we'll create will:
1. Receive the initial article request from the user
2. Delegate article creation to the Article Generation Agent
3. Send the draft to the Article Reviewer Agent for critique
4. Return the draft to the Article Generation Agent with feedback
5. Repeat steps 2-4 up to three times to refine the article
6. Deliver the final polished article to the user
   
The following architecture diagram illustrates this multi-agent collaboration workflow:

<img src="../imgs/article_generation_supervisor_agent.png" width="600">

Let's get started!

First we restore the variables from the previous notebook

In [1]:
%store -r

## Create a Article Generation Supervisor agent using Amazon Bedrock Agent
On this section we declare global variables that will be act as helpers during entire notebook.

In [ ]:
import boto3
import json
import uuid

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-pro-v1:0'
]

## Importing helper functions
On following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, knowledge bases for bedrock creation and accessing data on S3.

In [3]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)

agents = AgentsForAmazonBedrock()

## Article Generation Supervisor Agent
Create a supervisor agent to connect the Artcle Generation and Reviewer agents to iteratively improve the quality of the article. 

We will provide the following instructions to our agent:
```
You are a supervisor AI agent. You are given 2 agents capable of following tasks:

1. Article Generation agent - Writes a draft of article based on the given instructions and/or recommendations.

2. Article Reviewer agent - Reviews the article generated by the Article Generation agent to provide recommendation to improve the article.

You are to leverage both the agents to work collaboratively and iteratively to arrive at the best version of the article. 

Here are the guidelines:
- You should only iterate the draft and review iterations for maximum of 3 time before returning the final version of the article to the user. 

The final draft of the article must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final article in markdown format.
```

In [4]:
article_supervisor_agent_name = "article_supervisor_agent"
supervisor_agent = agents.create_agent(
    article_supervisor_agent_name,
    """You are a supervisor AI agent that creates a high quality PR articles based on user input. 
""",
    """You are a supervisor AI agent. You are given 2 agents capable of following tasks:

1. Article Generation agent - Writes a draft of article based on the given instructions and/or recommendations.

2. Article Reviewer agent - Reviews the article generated by the Article Generation agent to provide recommendation to improve the article.

You are to leverage both the agents to work collaboratively and iteratively to arrive at the best version of the article. 

Here are the guidelines:
- You should only iterate the draft and review iterations for maximum of 3 time before returning the final version of the article to the user. 

The final draft of the article must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final article in markdown format.
""",
    agent_foundation_model,
    agent_collaboration='SUPERVISOR'
)

In [5]:
article_supervisor_agent_id = supervisor_agent[0]

### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside energy one (which is multi-agent collaborator).

In [6]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': article_generation_agent_alias_arn,
        'sub_agent_instruction': """Delegate article writing task to the Article Generation Agent, ensuring adherence to its specific protocols and capabilities.""",
        'sub_agent_association_name': 'ArticleGenerationAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': article_reviewer_agent_alias_arn,
        'sub_agent_instruction': """Delegate article review task to the Article Reviewer Agent, ensuring most helpful feedback is given by the agent.""",
        'sub_agent_association_name': 'ArticleReviewerAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Associate all sub-agents with this one (main one).

In [ ]:
article_supervisor_agent_alias_id, article_supervisor_agent_alias_arn = agents.associate_sub_agents(
    article_supervisor_agent_id, sub_agents_list
)

## Invoking multi-agent Collaborator

With our supervisor agent created and connected to both specialized agents, we can now demonstrate the power of multi-agent collaboration. The supervisor agent will orchestrate the entire workflow:

1. It will first delegate the article creation task to the Article Generation Agent
2. Then send the draft to the Article Reviewer Agent for detailed feedback
3. Return the draft with feedback to the Article Generation Agent for refinement
4. Repeat this cycle up to three times to continuously improve the article quality
5. Finally deliver the polished article to the user

This collaborative approach combines the strengths of both specialized agents - the creative writing capabilities of the Article Generation Agent and the critical analysis skills of the Article Reviewer Agent - resulting in higher quality content than either agent could produce independently.

We can now set a dictionary with agents names for better visualization of the traces

In [8]:
multi_agent_names = {
    f"{article_generation_agent_id}/{article_generation_alias_id}": article_generation_agent_name,
    f"{article_reviewer_agent_id}/{article_reviewer_agent_alias_id}": article_reviewer_agent_name,
    f"{article_supervisor_agent_id}/{article_supervisor_agent_alias_id}": article_supervisor_agent_name
}

Let's test this multi-agent system by providing a prompt to create a high-quality article based on specific requirements.

In [ ]:
# Load the research data from the JSON file
with open('sample_output_from_research.json', 'r') as f:
    research_data = json.load(f)

# Format the research data for the prompt
research_details = "Research Information:\n"
for entity in research_data["entities"]:
    research_details += f"\n{entity['name']} ({entity['type']}):\n"
    for bg in entity['background']:
        research_details += f"- {bg}\n"

print(research_details)

In [ ]:
article_query_prompt = f"""
Write a business news article about NeuraHealth Solutions and their AI-driven healthcare innovations based on the following research data:

{research_details}

Focus on the company's leadership, their products (especially MediScan), and the regulatory environment they operate in. 
Include quotes from the executives and discuss the company's market position and future prospects.
The article should be informative, balanced, and suitable for a business publication.
"""

article_query_prompt

In [ ]:
%%time
session_id:str = str(uuid.uuid4())


article_supervisor_agent_response = agents.invoke(
    article_query_prompt,
    article_supervisor_agent_id,
    session_id=session_id,
    enable_trace=True,
    trace_level = "all",
    multi_agent_names=multi_agent_names
)

In [ ]:
print(article_supervisor_agent_response)

# Summary

In this lab, we demonstrated how to implement a powerful **Multi-Agent Collaboration** pattern using Amazon Bedrock Agents. This pattern enables complex workflows through the orchestration of specialized agents working together to achieve superior results.

Our implementation consisted of three key components:

1. **Article Generation Agent**: A specialized content creator that produces high-quality articles following detailed journalistic guidelines across multiple dimensions including clarity, accuracy, engagement, structure, and technical excellence.

2. **Article Reviewer Agent**: A professional reviewer that analyzes content across multiple dimensions and provides detailed, constructive feedback with specific improvement suggestions.

3. **Article Supervisor Agent**: The orchestrator that coordinates the collaboration between the specialized agents, managing the iterative feedback loop to continuously refine the article quality.

The multi-agent collaboration pattern demonstrated several key advantages:

- **Specialized Expertise**: Each agent focuses on its core competency - content creation or critical review
- **Iterative Improvement**: The feedback loop enables continuous refinement of the content
- **Workflow Automation**: The supervisor agent manages the entire process without requiring user intervention
- **Quality Enhancement**: The final output benefits from multiple perspectives and expertise

This pattern can be extended to many other domains where complex workflows require different specialized capabilities working in concert. By breaking down complex tasks into specialized roles and establishing clear communication protocols between agents, we can build powerful AI systems that deliver higher quality results than any single agent could achieve independently.


## Saving information
Let's store the environment variables to be used in other notebooks.

In [ ]:
%store article_supervisor_agent_alias_id
%store article_supervisor_agent_alias_arn
%store article_supervisor_agent_id
%store article_supervisor_agent_name